# Data Munging with PySpark

### <font color='green'>_This notebook supports Google Colab_  </font>

<font color='green'>Look for the "_Sidebar_: Google Colab" section below to setup and run this Spark notebook on Google Colab.</font>

# Installation etc.

Look, I hate to be that guy, but Google/DDG are your friends here.  
Getting ```Spark```, ```pySpark``` and other libraries to run is more than a little tedious.  
The following are a 'best-guess' set of instrustions.  
The ones that worked for me.  
Using Windows 10 here.  
Your mileage may vary.  

Once you are through the tedium of installation and setup - it gets good. I promise. :)  

## Download (and install where applicable)
* JDK (prefer 8.x/11.x, 64bit, more open the better)
* Hadoop (3.2.x, at this time) - _for windows, we just need Hadoop Winutils_
* [Hadoop *winutils* (corresponding to the version of Hadoop)](https://github.com/cdarlint/winutils), [another repo](https://github.com/kontext-tech/winutils)
* [Spark (3.x, at this time)](https://spark.apache.org/downloads.html)  
* [Anaconda - Open Source/Individual Edition](https://www.anaconda.com/products/distribution)

## Setup environment variables 

We set these environment variables that help manage paths better.
Example variable values would look like:
Java:  
* JAVA_HOME = ```C:\[Java]```  
    
Hadoop:  
* HADOOP_HOME = ```C:\hadoop\hadoop-3.4.0-win10-x64```  
_this is just a sample_  
_On Windows, we need HADOOP_HOME to be the folder where **winutils.exe** is located_  
_For dev purposes: you can just download winutils and move on, just ensure you point to winutils.exe in this env variable_

finally, Spark:   
* SPARK_HOME = ```C:\Spark\spark-3.4.1-bin-hadoop3```  

*notice there are no backslashes in the end. This is because slashes will be added in the next step when we setup path*      

## Update system **'PATH'**

We use the variables defined above to set-up paths.  

* Java: ```%JAVA_HOME%/bin```
* Hadoop 01: ```%HADOOP_HOME%/bin``` 
* Hadoop 02: ```%HADOOP_HOME%/sbin``` (*sbin needed in addition to bin*)
* Spark: ```%SPARK_HOME%/bin```  
    
(*here we add backslashes before bin*)

N.B.: If you are only doing the dev setup (just winutils and nothing else) thing, you may not need the Hadoop 01/02 above, I added those to my system anyway.

## Patch Hadoop  

This is *needed* when Hadoop is run on Windows.

* copy the ```bin``` folder from the right version of winutils to replace ```%HADOOP_HOME%/bin```  

* copy ```hadoop-yarn-server-timelineservice-3.0.3``` from ```%HADOOP_HOME%\share\hadoop\yarn\timelineservice``` to ```%HADOOP_HOME%\share\hadoop\yarn``` (the parent directory).  

## Install the Python libraries

Prefer installing [Anaconda](https://www.anaconda.com/products/distribution). 
It resolves other dependencies like Pandas, Numpy, Jupyter etc. too.  
Once there, use either pip or conda - they are both cool but incompatible.  
The conda-forge channel is a few days behind the pip one.  
We're only running on the local machine here, no complicated infrastructure to care about.  
So, you do you.  

Use one of the following commands (from the command line obvs) to install each:  
* pyspark:
    * ```pip install pyspark``` or
    * ```conda install -c conda-forge pyspark```
* findspark:
    * ```pip install findspark``` or
    * ```conda install -c conda-forge findspark```

## References  

* [How to install Hadoop on Win 10](https://muhammadbilalyar.github.io/blogs/How-to-install-Hadoop-on-Window-10/)
* [Hadoop on Windows](https://github.com/MuhammadBilalYar/Hadoop-On-Window)
* [Hadoop and Spark on Windows](https://dev.to/awwsmm/installing-and-running-hadoop-and-spark-on-windows-33kc)

### (_Optionally_) Configure Hadoop

*only needed if you want to use hadoop as your file storage system*  

* create a folder for ```namenode```
* create a folder for ```datanode```
* four files: ```core-site.xml```, ```mapred-site.xml```, ```hdfs-site.xml```, ```yarn-site.xml``` - see code for each in the [reference repo](https://github.com/MuhammadBilalYar/Hadoop-On-Window) above.

### Also,

The scope of this notebook is *usage* - not setup or troubleshooting, am pretty sure these installation instructions will be outdated soon and be replaced by pre-built docker images or shell scripts or automated installs for windows or such-like.  

# Setup

This boiler plate helps, esp. in Jupyter Notebook situations

## _Sidebar_: Google Colab

You don't need to run this on your local machine.
The notebook is setup to run on Google Colab as well.

For a detailed description of how this is setup, see the [02.000 (optional) Setup_Spark_in_Google_Colab](https://github.com/shauryashaurya/learn-data-munging/blob/main/02.000%20(optional)%20Setup_Spark_in_Google_Colab.ipynb) notebook

Open the notebook in Google Colab using the following button, then uncomment the setup marked # SETUP FOR COLAB  
  
  
<a href="https://colab.research.google.com/github/shauryashaurya/learn-data-munging/blob/main/02.001%20-%2010%2B%20minutes%20to%20pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>    

_NOTE: keep the # SETUP FOR COLAB step below commented (disabled) when you are running this notebook locally_

In [1]:
# SETUP FOR COLAB: select all the lines below and uncomment (CTRL+/ on windows)

# # grab spark
# # as of Dec 2022, the latest version is 3.2.3, get the link from Apache Spark's website
# ! wget -q https://dlcdn.apache.org/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz
# # unzip spark
# !tar xf spark-3.2.3-bin-hadoop3.2.tgz
# # install findspark package
# !pip install -q findspark

# # got to provide JAVA_HOME and SPARK_HOME vairables
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop3.2"

In [2]:
# Step 1: initialize findspark
import findspark
findspark.init()

In [3]:
# Step 2: import pyspark
import pyspark
from pyspark.sql import SparkSession

In [4]:
pyspark.__version__

'3.4.1'

In [5]:
# Step 3: Create a spark session

# 'local[1]' indicates spark on 1 core on the local machine, specify the number of cores needed
# use .config("spark.some.config.option", "some-value") for additional configuration

spark = SparkSession \
    .builder \
    .master('local[1]') \
    .appName("10+ minutes to pyspark") \
    .getOrCreate()

# spark

Back in the day you'd need various 'contexts' as entry points into spark functionality.  
All of this is now wrapped into a SparkSession, easy to manage.

In [6]:
# The SparkSession carries the sparkContext
# spark.sparkContext

Check out the spark UI link when you uncomment the lines in the two cells above.  
Your local UI should launch at a link like: http://localhost:4041/jobs/

In [7]:
# before we close the notebook, stop spark, otherwise Jupyter closes, but scala-spark keep going on...
# spark.stop()

# Dataframes

## DataFrames: Create and View

In [8]:
from datetime import datetime, date
import numpy as np
import pandas as pd
from pyspark.sql import Row

In [9]:
# use a list of pyspark.sql.Row
df1 = spark.createDataFrame(
    [
        Row(a=1,b=2.,c='span a',d=date(2022,7,1),e=datetime(2022,7,1,12,0)),
        Row(a=1,b=3.,c='can a ',d=date(2022,7,2),e=datetime(2022,7,2,12,0,1)),
        Row(a=1,b=4.,c='banana',d=date(2022,7,3),e=datetime(2022,7,3,12,0,2))
    ]
)

In [10]:
df1

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [11]:
# df1's not been evaluated yet. It's lazy evaluation
# to eval it, we go
df1.show()

+---+---+------+----------+-------------------+
|  a|  b|     c|         d|                  e|
+---+---+------+----------+-------------------+
|  1|2.0|span a|2022-07-01|2022-07-01 12:00:00|
|  1|3.0|can a |2022-07-02|2022-07-02 12:00:01|
|  1|4.0|banana|2022-07-03|2022-07-03 12:00:02|
+---+---+------+----------+-------------------+



In [12]:
# use a list of tuples with explicit schema
df2 = spark.createDataFrame(
    [
        (2,5.,'man a',date(2022,7,1),datetime(2022,7,1,12,0)),
        (2,6.,'can a',date(2022,8,1),datetime(2022,7,2,12,0)),
        (2,7.,'manna',date(2022,9,1),datetime(2022,7,3,12,0))
    ],
    schema = 'a bigint, b double, c string, d date, e timestamp'
)
df2

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [13]:
# use spark RDDs to create a dataframe
# go to the sparksession's sparkContext to access parallelize method
rdd3 = spark.sparkContext.parallelize(
    [
        (3,5., 'main', date(2022,7,1), datetime(2022,7,1,12,0,1)),
        (3,5., 'brain', date(2022,7,1), datetime(2022,7,1,12,0,1)),
        (3,5., 'pain', date(2022,7,1), datetime(2022,7,1,12,0,1))
    ]
)

df3 = spark.createDataFrame(rdd3, schema=['a','b','c','d','e'])

df3

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [14]:
# can also use a pandas dataframe to create a spark dataframe
df4_pd = pd.DataFrame(
    {
        'a': np.random.randint(0,10, size = 3),
        'b': np.random.randn(3),
        'c': ["gandalf's manager", "said", 'no'],
        'd': [date(2022,7,1),date(2022,7,2),date(2022,7,3)],
        'e': [datetime(2022,7,1,12,0,1),datetime(2022,7,2,12,0,2),datetime(2022,7,3,12,0,3)]
    }
)

df4 = spark.createDataFrame(df4_pd)

df4.show()

+---+--------------------+-----------------+----------+-------------------+
|  a|                   b|                c|         d|                  e|
+---+--------------------+-----------------+----------+-------------------+
|  0|0.019948190425159388|gandalf's manager|2022-07-01|2022-07-01 12:00:01|
|  0|  1.2033849932496083|             said|2022-07-02|2022-07-02 12:00:02|
|  2| -0.4732124870595125|               no|2022-07-03|2022-07-03 12:00:03|
+---+--------------------+-----------------+----------+-------------------+



In [15]:
df4_pd

,a,b,c,d,e
0,0,0.019948,gandalf's manager,2022-07-01,2022-07-01 12:00:01
1,0,1.203385,said,2022-07-02,2022-07-02 12:00:02
2,2,-0.473212,no,2022-07-03,2022-07-03 12:00:03


In [16]:
# use printSchema() to..., you know, it says what it does
# also don't you hate that pySpark is following the Java camelCase instead of the Python snake_case?
# yeah, what's that all about?

df1.printSchema()
df2.printSchema()
df3.printSchema()
df4.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [17]:
# show only x rows
df1.show(1)
# vertical - if the row is too long for horizontal display
df4.show(2, vertical=True)

+---+---+------+----------+-------------------+
|  a|  b|     c|         d|                  e|
+---+---+------+----------+-------------------+
|  1|2.0|span a|2022-07-01|2022-07-01 12:00:00|
+---+---+------+----------+-------------------+
only showing top 1 row

-RECORD 0-------------------
 a   | 0                    
 b   | 0.019948190425159388 
 c   | gandalf's manager    
 d   | 2022-07-01           
 e   | 2022-07-01 12:00:01  
-RECORD 1-------------------
 a   | 0                    
 b   | 1.2033849932496083   
 c   | said                 
 d   | 2022-07-02           
 e   | 2022-07-02 12:00:02  
only showing top 2 rows



In [18]:
# collect() - collects the entire df from across all nodes to the driver
# if you don't have enough memory, here's how you crash spark
# careful is the word
df3.collect()

[Row(a=3, b=5.0, c='main', d=datetime.date(2022, 7, 1), e=datetime.datetime(2022, 7, 1, 12, 0, 1)),
 Row(a=3, b=5.0, c='brain', d=datetime.date(2022, 7, 1), e=datetime.datetime(2022, 7, 1, 12, 0, 1)),
 Row(a=3, b=5.0, c='pain', d=datetime.date(2022, 7, 1), e=datetime.datetime(2022, 7, 1, 12, 0, 1))]

In [19]:
# Pandas used to have a take() method, deprecated now
# take() in spark extracts the first n rows of a dataframe
df2.take(2)

[Row(a=2, b=5.0, c='man a', d=datetime.date(2022, 7, 1), e=datetime.datetime(2022, 7, 1, 12, 0)),
 Row(a=2, b=6.0, c='can a', d=datetime.date(2022, 8, 1), e=datetime.datetime(2022, 7, 2, 12, 0))]

In [20]:
# same as take() but returns the last n rows of a dataframe
df2.tail(2)

[Row(a=2, b=6.0, c='can a', d=datetime.date(2022, 8, 1), e=datetime.datetime(2022, 7, 2, 12, 0)),
 Row(a=2, b=7.0, c='manna', d=datetime.date(2022, 9, 1), e=datetime.datetime(2022, 7, 3, 12, 0))]

In [21]:
# hey you want another way of crashing the spark driver?
# convert the spark dataframe to a pandas dataframe
# it'll collect all data from all workers into the driver

# this naive approach will fail because we have date-time values in df3 . 
# df3.toPandas()
# possible error: TypeError: Casting to unit-less dtype 'datetime64' is not supported. Pass e.g. 'datetime64[ns]' instead.

# explicitly provide date-time data type to Pandas
from pyspark.sql.functions import date_format
df3.withColumn("d", date_format("d", "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("e", date_format("e", "yyyy-MM-dd HH:mm:ss")) \
    .toPandas()

,a,b,c,d,e
0,3,5.0,main,2022-07-01 00:00:00,2022-07-01 12:00:01
1,3,5.0,brain,2022-07-01 00:00:00,2022-07-01 12:00:01
2,3,5.0,pain,2022-07-01 00:00:00,2022-07-01 12:00:01


In [22]:
# how do I make it so my dataframes are evaluated eagerly?
# instead of the regular lazy eval?
# y'know when am in notebooks and stuff?
# set the config
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [23]:
# there you go, eager evaulatio'
df1

a,b,c,d,e
1,2.0,span a,2022-07-01,2022-07-01 12:00:00
1,3.0,can a,2022-07-02,2022-07-02 12:00:01
1,4.0,banana,2022-07-03,2022-07-03 12:00:02


In [24]:
# set it back to false if you like
spark.conf.set('spark.sql.repl.eagerEval.enabled', False)

In [25]:
# No more eagerly evaluated dataframes
df1

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

# Selecting and Accessing Data

In [26]:
# access a column
df1.a, df2.b, df3.c, df4.d

(Column<'a'>, Column<'b'>, Column<'c'>, Column<'d'>)

In [27]:
from pyspark.sql import Column
from pyspark.sql.functions import upper

In [28]:
type(df1.c) == type(upper(df1.c)) == type(df1.c.isNull())
# TODO: what's going on with type(df1.c.isNull()) above???

True

In [29]:
df1.c.isNull()

Column<'(c IS NULL)'>

In [30]:
# use dataframe's select method to identify a column and show() it
df1.select(df1.c).show()

+------+
|     c|
+------+
|span a|
|can a |
|banana|
+------+



In [31]:
# also there's dataframe.filter()
df4.filter(df4.a>0).show()

+---+-------------------+---+----------+-------------------+
|  a|                  b|  c|         d|                  e|
+---+-------------------+---+----------+-------------------+
|  2|-0.4732124870595125| no|2022-07-03|2022-07-03 12:00:03|
+---+-------------------+---+----------+-------------------+



In [32]:
# spark.stop()

In [33]:
# assign a new column instance to the dataframe
df4_withNewCol = df4.withColumn('upper_c', upper(df4.c))
df4_withNewCol.show()

+---+--------------------+-----------------+----------+-------------------+-----------------+
|  a|                   b|                c|         d|                  e|          upper_c|
+---+--------------------+-----------------+----------+-------------------+-----------------+
|  0|0.019948190425159388|gandalf's manager|2022-07-01|2022-07-01 12:00:01|GANDALF'S MANAGER|
|  0|  1.2033849932496083|             said|2022-07-02|2022-07-02 12:00:02|             SAID|
|  2| -0.4732124870595125|               no|2022-07-03|2022-07-03 12:00:03|               NO|
+---+--------------------+-----------------+----------+-------------------+-----------------+



In [34]:
df4.show()
df4_withNewCol.show()

+---+--------------------+-----------------+----------+-------------------+
|  a|                   b|                c|         d|                  e|
+---+--------------------+-----------------+----------+-------------------+
|  0|0.019948190425159388|gandalf's manager|2022-07-01|2022-07-01 12:00:01|
|  0|  1.2033849932496083|             said|2022-07-02|2022-07-02 12:00:02|
|  2| -0.4732124870595125|               no|2022-07-03|2022-07-03 12:00:03|
+---+--------------------+-----------------+----------+-------------------+

+---+--------------------+-----------------+----------+-------------------+-----------------+
|  a|                   b|                c|         d|                  e|          upper_c|
+---+--------------------+-----------------+----------+-------------------+-----------------+
|  0|0.019948190425159388|gandalf's manager|2022-07-01|2022-07-01 12:00:01|GANDALF'S MANAGER|
|  0|  1.2033849932496083|             said|2022-07-02|2022-07-02 12:00:02|             SAI

In [35]:
# filter
df4.filter(df4.a == 9).show()

+---+---+---+---+---+
|  a|  b|  c|  d|  e|
+---+---+---+---+---+
+---+---+---+---+---+



In [36]:
df4.filter(df4.b > 0).show()

+---+--------------------+-----------------+----------+-------------------+
|  a|                   b|                c|         d|                  e|
+---+--------------------+-----------------+----------+-------------------+
|  0|0.019948190425159388|gandalf's manager|2022-07-01|2022-07-01 12:00:01|
|  0|  1.2033849932496083|             said|2022-07-02|2022-07-02 12:00:02|
+---+--------------------+-----------------+----------+-------------------+



# UDFs: Applying a Function

In [37]:


import pandas as pd
from pyspark.sql.functions import pandas_udf

@pandas_udf('long')
def pandas_plus_one(series: pd.Series) -> pd.Series:
    #     plus one using pandas series
    return series+1

df4.select(df4.a, pandas_plus_one(df4.a)).show()

+---+------------------+
|  a|pandas_plus_one(a)|
+---+------------------+
|  0|                 1|
|  0|                 1|
|  2|                 3|
+---+------------------+



In [38]:
def pandas_filter(iterator):
    for pandas_df in iterator:
        yield pandas_df[pandas_df.b>0]
        
df4.mapInPandas(pandas_filter, schema = df4.schema).show()

+---+--------------------+-----------------+----------+-------------------+
|  a|                   b|                c|         d|                  e|
+---+--------------------+-----------------+----------+-------------------+
|  0|0.019948190425159388|gandalf's manager|2022-07-01|2022-07-01 12:00:01|
|  0|  1.2033849932496083|             said|2022-07-02|2022-07-02 12:00:02|
+---+--------------------+-----------------+----------+-------------------+



# Grouping Data

Split-Apply-Combine, just like pandas

In [39]:
# group by fruit, color etc.
df5 = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], schema=['color', 'fruit', 'v1', 'v2'])

df5.show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



In [40]:
df5.groupby('color').avg().show()

+-----+-------+-------+
|color|avg(v1)|avg(v2)|
+-----+-------+-------+
|  red|    4.8|   48.0|
|black|    6.0|   60.0|
| blue|    3.0|   30.0|
+-----+-------+-------+



In [41]:
# TODO: How to get the deviations in a new column?
def plus_mean(pandas_df):
    return pandas_df.assign(v1=pandas_df.v1 - pandas_df.v1.mean())

df5.groupby('color').applyInPandas(plus_mean, schema = df5.schema).show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|black|carrot|  0| 60|
| blue|banana| -1| 20|
| blue| grape|  1| 40|
|  red|banana| -3| 10|
|  red|carrot| -1| 30|
|  red|carrot|  0| 50|
|  red|banana|  2| 70|
|  red| grape|  3| 80|
+-----+------+---+---+



In [42]:
# Co-grouping and applying a function.

df6 = spark.createDataFrame(
    [
        (20000101, 1, 1.0), 
        (20000101, 2, 2.0), 
        (20000102, 1, 3.0), 
        (20000102, 2, 4.0)
    ],
    ('time', 'id', 'v1')
)

df7 = spark.createDataFrame(
    [
        (20000101, 1, 'x'), 
        (20000101, 2, 'y')
    ],
    ('time', 'id', 'v2')
)

In [43]:
def asof_join(l, r):
    return pd.merge_asof(l,r,on='time', by='id')

In [44]:
df6_gb = df6.groupby('id')
df7_gb = df7.groupby('id')

In [45]:
co_grp = df6_gb.cogroup(df7_gb)

In [46]:
rslt = co_grp.applyInPandas(asof_join, schema='time int, id int, v1 double, v2 string')
rslt.show()

+--------+---+---+---+
|    time| id| v1| v2|
+--------+---+---+---+
|20000101|  1|1.0|  x|
|20000102|  1|3.0|  x|
|20000101|  2|2.0|  y|
|20000102|  2|4.0|  y|
+--------+---+---+---+



# Getting data in and out

In [47]:
# CSV
# .mode('overwrite') can be taken out when needed.
df5.write.mode('overwrite').csv('fruits.csv', header=True)
spark.read.csv('fruits.csv', header=True).show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



In [48]:
# Parquet
# .mode('overwrite') can be taken out when needed.
df5.write.mode('overwrite').parquet('fruits.parquet')
spark.read.parquet('fruits.parquet').show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



# Working with SQL

In [49]:
df5.createOrReplaceTempView('tableA')

In [50]:
spark.sql('select count(*) from tableA').show()

+--------+
|count(1)|
+--------+
|       8|
+--------+



In [51]:
# UDFs in SQL
# register and invoke
@pandas_udf('integer')
def add_one(s: pd.Series) -> pd.Series:
    return s+1

spark.udf.register('add_one', add_one)

<function __main__.add_one(s: pandas.core.series.Series) -> pandas.core.series.Series>

In [52]:
spark.sql('SELECT v1, add_one(v1) from tableA').show()

+---+-----------+
| v1|add_one(v1)|
+---+-----------+
|  1|          2|
|  2|          3|
|  3|          4|
|  4|          5|
|  5|          6|
|  6|          7|
|  7|          8|
|  8|          9|
+---+-----------+



In [53]:
# can mix/match sql expressions 
# for e.g. take the expressions from above

from pyspark.sql.functions import expr

df5.selectExpr('add_one(v1)').show()

+-----------+
|add_one(v1)|
+-----------+
|          2|
|          3|
|          4|
|          5|
|          6|
|          7|
|          8|
|          9|
+-----------+



In [54]:
df5.select(expr('count(*)')).show()

+--------+
|count(1)|
+--------+
|       8|
+--------+



In [55]:
df5.select(expr('count(*)')>0).show()

+--------------+
|(count(1) > 0)|
+--------------+
|          true|
+--------------+



# Pandas API on Spark

In [56]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession

C:\1\spark-3.4.1-bin-hadoop3\python\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [57]:
# create a pandas on spark series

s1 = ps.Series([1,2,3,np.nan,5,np.nan,7,8,9])
s1

0    1.0
1    2.0
2    3.0
3    NaN
4    5.0
5    NaN
6    7.0
7    8.0
8    9.0
dtype: float64

In [58]:
psdf1 = ps.DataFrame(
    {'a': [1, 2, 3, 4, 5, 6],
     'b': [100, 200, 300, 400, 500, 600],
     'c': ["one", "two", "three", "four", "five", "six"]
    },
    index=[10, 20, 30, 40, 50, 60]
)

psdf1

,a,b,c
10,1,100,one
20,2,200,two
30,3,300,three
40,4,400,four
50,5,500,five
60,6,600,six


In [59]:
# create a pandas dataframe and convert to pandas-spark
dates1 = pd.date_range('20220101', periods=6)

In [60]:
pdf1 = pd.DataFrame(np.random.randn(6,4), index=dates1, columns = list('ABCD'))


In [61]:
pdf1

,A,B,C,D
2022-01-01,0.587632,0.416540,-0.994577,-0.253461
2022-01-02,0.079831,1.135812,0.198541,0.868861
2022-01-03,1.889186,2.138279,0.822015,0.172719
2022-01-04,0.013381,-0.629434,-0.760277,-0.334733
2022-01-05,-1.074110,0.535503,-1.980942,-0.553443
2022-01-06,0.490397,-2.092646,-0.073376,0.313900


In [62]:
pdf1.index

DatetimeIndex(['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04',
               '2022-01-05', '2022-01-06'],
              dtype='datetime64[ns]', freq='D')

In [63]:
# create spark dataframe from the pandas dataframe

# naive method breaks with pandas 2.x due to date-time column
# psdf1 = ps.from_pandas(pdf1)

# we follow a 2-step approach here:
# first convert datetime to string
pdf1.index = pdf1.index.astype("string")
psdf1 = ps.from_pandas(pdf1)

# now cast index back to datetime64 
psdf1.index = psdf1.index.astype("datetime64")

C:\1\spark-3.4.1-bin-hadoop3\python\pyspark\pandas\frame.py:13540: UserWarning: pandas-on-Spark doesn't allow columns to be created via a new attribute name
  warnings.warn(msg, UserWarning)


In [64]:
# this is a pandas-on-spark dataframe now
type(psdf1)

pyspark.pandas.frame.DataFrame

In [65]:
psdf1

,A,B,C,D
2022-01-01,0.587632,0.416540,-0.994577,-0.253461
2022-01-02,0.079831,1.135812,0.198541,0.868861
2022-01-03,1.889186,2.138279,0.822015,0.172719
2022-01-04,0.013381,-0.629434,-0.760277,-0.334733
2022-01-05,-1.074110,0.535503,-1.980942,-0.553443
2022-01-06,0.490397,-2.092646,-0.073376,0.313900


In [66]:
# create a SPARK df from a PANDAS df (we've done this before)
# create a PANDAS-ON-SPARK df from a SPARK df (we've done this too...)

sdf2 = spark.createDataFrame(pdf1)
sdf2.show()

+--------------------+-------------------+--------------------+--------------------+
|                   A|                  B|                   C|                   D|
+--------------------+-------------------+--------------------+--------------------+
|  0.5876316884770005|0.41654012275843466| -0.9945770872112353|-0.25346099450704385|
| 0.07983090136415741| 1.1358122984861798|  0.1985413431488227|  0.8688613503299467|
|   1.889186412861359| 2.1382789477538484|  0.8220153761951643| 0.17271927746171553|
|0.013381003884591118|-0.6294338590575285| -0.7602769001052844|-0.33473265367613747|
|  -1.074109545152633| 0.5355031079015387| -1.9809420819529149| -0.5534427441521881|
|  0.4903970254280107|-2.0926463858000153|-0.07337612011013382|  0.3139002370789783|
+--------------------+-------------------+--------------------+--------------------+



In [67]:
psdf2 = sdf2.pandas_api()
# this works like a pandas df now...
psdf2

,A,B,C,D
0,0.587632,0.416540,-0.994577,-0.253461
1,0.079831,1.135812,0.198541,0.868861
2,1.889186,2.138279,0.822015,0.172719
3,0.013381,-0.629434,-0.760277,-0.334733
4,-1.074110,0.535503,-1.980942,-0.553443
5,0.490397,-2.092646,-0.073376,0.313900


In [68]:
type(psdf2)

pyspark.pandas.frame.DataFrame

In [69]:
psdf2.dtypes

A    float64
B    float64
C    float64
D    float64
dtype: object

In [70]:
# pandas head()
psdf2.head()

,A,B,C,D
0,0.587632,0.416540,-0.994577,-0.253461
1,0.079831,1.135812,0.198541,0.868861
2,1.889186,2.138279,0.822015,0.172719
3,0.013381,-0.629434,-0.760277,-0.334733
4,-1.074110,0.535503,-1.980942,-0.553443


In [71]:
psdf2.index

Index([0, 1, 2, 3, 4, 5], dtype='int64')

In [72]:
psdf2.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [73]:
psdf2.describe()

,A,B,C,D
count,6.000000,6.000000,6.000000,6.000000
mean,0.331053,0.250676,-0.464769,0.035641
std,0.965144,1.464302,0.991543,0.521201
min,-1.074110,-2.092646,-1.980942,-0.553443
25%,0.013381,-0.629434,-0.994577,-0.334733
50%,0.079831,0.416540,-0.760277,-0.253461
75%,0.587632,1.135812,0.198541,0.313900
max,1.889186,2.138279,0.822015,0.868861


In [74]:
# warning - this breaks the driver due to memory - just like collect()
# be careful
psdf2.to_numpy()

C:\1\spark-3.4.1-bin-hadoop3\python\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


array([[ 0.58763169,  0.41654012, -0.99457709, -0.25346099],
       [ 0.0798309 ,  1.1358123 ,  0.19854134,  0.86886135],
       [ 1.88918641,  2.13827895,  0.82201538,  0.17271928],
       [ 0.013381  , -0.62943386, -0.7602769 , -0.33473265],
       [-1.07410955,  0.53550311, -1.98094208, -0.55344274],
       [ 0.49039703, -2.09264639, -0.07337612,  0.31390024]])

In [75]:
# transpose
psdf2.T

,0,1,2,3,4,5
A,0.587632,0.079831,1.889186,0.013381,-1.074110,0.490397
B,0.416540,1.135812,2.138279,-0.629434,0.535503,-2.092646
C,-0.994577,0.198541,0.822015,-0.760277,-1.980942,-0.073376
D,-0.253461,0.868861,0.172719,-0.334733,-0.553443,0.313900


In [76]:
# sort index
psdf2.sort_index(ascending = False)

,A,B,C,D
5,0.490397,-2.092646,-0.073376,0.313900
4,-1.074110,0.535503,-1.980942,-0.553443
3,0.013381,-0.629434,-0.760277,-0.334733
2,1.889186,2.138279,0.822015,0.172719
1,0.079831,1.135812,0.198541,0.868861
0,0.587632,0.416540,-0.994577,-0.253461


In [77]:
# sort values
psdf2.sort_values(by='B', ascending = True)

,A,B,C,D
5,0.490397,-2.092646,-0.073376,0.313900
3,0.013381,-0.629434,-0.760277,-0.334733
0,0.587632,0.416540,-0.994577,-0.253461
4,-1.074110,0.535503,-1.980942,-0.553443
1,0.079831,1.135812,0.198541,0.868861
2,1.889186,2.138279,0.822015,0.172719


# Missing Data - Pandas on Spark

In [78]:
# you'll notice we are repeating a lot of the tasks 
# from 10 minutes to pandas 
pdf2 = pdf1.reindex(index=dates1[0:4], columns=list(pdf1.columns) + ['E'])

In [79]:
pdf2.loc[dates1[0]:dates1[1], 'E'] = 1

In [80]:
# updated for pandas 2.x+
# explicitly cast dates to string
pdf2.index = pdf2.index.astype("string")
psdf3 = ps.from_pandas(pdf2)
# convert index back to datetime64
psdf3.index = psdf3.index.astype("datetime64")

C:\1\spark-3.4.1-bin-hadoop3\python\pyspark\pandas\frame.py:13540: UserWarning: pandas-on-Spark doesn't allow columns to be created via a new attribute name
  warnings.warn(msg, UserWarning)


In [81]:
psdf3

,A,B,C,D,E
2022-01-01,NaN,NaN,NaN,NaN,1.0
2022-01-02,NaN,NaN,NaN,NaN,1.0
2022-01-03,NaN,NaN,NaN,NaN,NaN
2022-01-04,NaN,NaN,NaN,NaN,NaN


In [82]:
psdf3.dropna(how='any')

,A,B,C,D,E


In [83]:
psdf3.fillna(value=5)

,A,B,C,D,E
2022-01-01,5.0,5.0,5.0,5.0,1.0
2022-01-02,5.0,5.0,5.0,5.0,1.0
2022-01-03,5.0,5.0,5.0,5.0,5.0
2022-01-04,5.0,5.0,5.0,5.0,5.0


# Operations - Pandas on Spark

## Stats

In [84]:
psdf1.mean()

A    0.331053
B    0.250676
C   -0.464769
D    0.035641
dtype: float64

## Spark Configurations

In [85]:
# store the current config value
previous = spark.conf.get('spark.sql.execution.arrow.pyspark.enabled')
previous

'false'

In [86]:
# Use default index prevent overhead.
ps.set_option('compute.default_index_type', 'distributed')

In [87]:
# ignore warnings coming from Arrow optimizations
import warnings
warnings.filterwarnings('ignore')

Let's toggle ```spark.sql.execution.arrow.pyspark.enabled``` to see the difference

In [88]:
# this is supposed to be faster by an order of magnitude
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)

In [89]:
%timeit ps.range(3000000).to_pandas()

1.18 s ± 511 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [90]:
# is supposed to be slower
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', False)

In [91]:
%timeit ps.range(3000000).to_pandas()

8.23 s ± 313 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [92]:
# put everything back as it was
ps.reset_option('compute.default_index_type')
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', previous)

# Grouping  
  
  
It's the same split-apply-combine deal.

In [93]:
psdf4 = ps.DataFrame({'A': ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                    'B': ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                    'C': np.random.randn(8),
                    'D': np.random.randn(8)})

In [94]:
psdf4

,A,B,C,D
0,foo,one,-1.051374,0.125254
1,bar,one,-0.149257,-0.816719
2,foo,two,-0.902433,0.740571
3,bar,three,0.477633,-0.179210
4,foo,two,-0.244086,1.524498
5,bar,two,-0.085297,-0.215543
6,foo,one,1.060077,0.591898
7,foo,three,0.633929,0.009752


In [95]:
psdf4.groupby('A').sum()

,C,D
A,,
bar,0.243079,-1.211472
foo,-0.503886,2.991972


In [96]:
psdf4.groupby(['A', 'B']).sum()

C         D
A   B                        
foo one    0.008703  0.717152
    two   -1.146519  2.265068
bar three  0.477633 -0.179210
foo three  0.633929  0.009752
bar two   -0.085297 -0.215543
    one   -0.149257 -0.816719

In [97]:
# same datetime nonsense with pandas v2.x is going to happen here...
pser = pd.Series(np.random.randn(1000),
                 index=pd.date_range('1/1/2000', periods=1000))

In [104]:
# ...so we cast the index as "string"
pser.index = pser.index.astype("string")
psser = ps.Series(pser)

In [105]:
psser = psser.cummax()

In [112]:
type(psser.index)

pyspark.pandas.indexes.base.Index

In [116]:
psser.reindex(psser.index.astype("string"))

2000-09-23    2.952296
2000-11-13    2.952296
2001-04-22    2.952296
2001-11-16    2.952296
2002-05-13    2.952296
2002-06-21    2.952296
2000-07-11    2.952296
2000-10-25    2.952296
2001-01-23    2.952296
2002-04-15    2.952296
2002-08-25    2.952296
2000-07-19    2.952296
2000-08-26    2.952296
2000-09-18    2.952296
2000-04-05    2.952296
2001-05-04    2.952296
2001-11-26    2.952296
2002-02-22    2.952296
2002-08-28    2.952296
2000-02-02    2.082675
2001-09-16    2.952296
2001-12-13    2.952296
2002-09-14    3.067316
2000-12-23    2.952296
2001-08-05    2.952296
2001-09-28    2.952296
2000-12-25    2.952296
2001-05-07    2.952296
2002-04-11    2.952296
2000-02-12    2.082675
2002-07-15    2.952296
2002-09-17    3.067316
2000-04-30    2.952296
2000-06-19    2.952296
2001-02-15    2.952296
2000-04-04    2.952296
2000-05-20    2.952296
2000-08-29    2.952296
2001-04-07    2.952296
2001-07-07    2.952296
2002-02-05    2.952296
2002-06-19    2.952296
2002-06-28    2.952296
2002-09-16 

In [117]:
psser.plot()

In [118]:
pdf4 = pd.DataFrame(np.random.randn(1000, 4), index=pser.index,
                   columns=['A', 'B', 'C', 'D'])

In [119]:
psdf5 = ps.from_pandas(pdf4)

In [120]:
psdf5 = psdf5.cummax()

In [121]:
psdf5.plot()

# Getting data in/out - Pandas on Spark

In [122]:
psdf4.to_csv('./data/foo.csv')
ps.read_csv('./data/foo.csv').head(4)

,A,B,C,D
0,foo,one,-1.051374,0.125254
1,bar,one,-0.149257,-0.816719
2,foo,two,-0.902433,0.740571
3,bar,three,0.477633,-0.179210


In [123]:
psdf4.to_parquet('./data/bar.parquet')
ps.read_parquet('./data/bar.parquet').head(3)

,A,B,C,D
0,foo,one,-1.051374,0.125254
1,bar,one,-0.149257,-0.816719
2,foo,two,-0.902433,0.740571


# All Done!

Let's stop spark now.

In [124]:
spark.stop()

# Up Next

Check out the exercises on real-world datasets next